# Louis Peyre Explorer - Colab (clean)
Notebook robuste pour exécuter le pipeline depuis Google Colab avec images sur Google Drive.

## 1) Runtime
Dans Colab: `Runtime > Change runtime type > GPU` (T4/L4/A100).

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content
!rm -rf /content/louis-peyre-explorer
!git clone https://github.com/gpeyre/louis-peyre-explorer.git
%cd /content/louis-peyre-explorer

## 2) Installer une stack PyTorch cohérente
Cette étape corrige l'erreur `operator torchvision::nms does not exist` en alignant `torch`, `torchvision`, `torchaudio`.

In [ ]:
!pip install -q --upgrade pip
!pip uninstall -y torch torchvision torchaudio transformers accelerate || true
!pip install -q --no-cache-dir --index-url https://download.pytorch.org/whl/cu124 torch==2.9.0 torchvision==0.24.0 torchaudio==2.9.0
!pip install -q --no-cache-dir -r requirements-colab.txt

In [ ]:
import torch, torchvision, transformers
print('torch       =', torch.__version__)
print('torchvision =', torchvision.__version__)
print('transformers=', transformers.__version__)
print('cuda available:', torch.cuda.is_available())
# Test crucial: l'opérateur nms doit exister
_ = torch.ops.torchvision.nms
print('OK: torch.ops.torchvision.nms disponible')

## 3) Vérifier les images Drive
Les images doivent être dans `/content/drive/MyDrive/louis-peyre-images`.

In [ ]:
!ls -la /content/drive/MyDrive/louis-peyre-images | head -n 40

## 4) Lancer le pipeline

In [ ]:
max_images = 20
sam_model_id = 'facebook/sam-vit-base'  # passe en sam-vit-huge si assez de VRAM

In [ ]:
!python pipeline.py \
  --drive_folder louis-peyre-images \
  --max_images {max_images} \
  --sam_model_id {sam_model_id}

## 5) Vérifier les sorties
Sorties attendues:
- `/content/drive/MyDrive/louis-peyre-images/crops/`
- `/content/drive/MyDrive/louis-peyre-images/provenance.csv`

In [ ]:
!ls -la /content/drive/MyDrive/louis-peyre-images/crops | head -n 40
!python - <<'PY'
import pandas as pd
p = '/content/drive/MyDrive/louis-peyre-images/provenance.csv'
df = pd.read_csv(p)
print(df.head())
print('Nombre de lignes:', len(df))
PY